# Lab #2B
___

**Task:** Train a multilayer perceptron for given dataset.

In [1]:
import numpy as np
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from misc.estimators import accuracy
from mlp import MLP

def decode(
        one_hot: np.ndarray
) -> np.ndarray:
    return np.argmax(one_hot, axis=1)

2021-11-06 15:13:38.619553: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-06 15:13:38.619588: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Dataset
___

We will use MNIST digits dataset and will try to predict
what digit is written on given image:

In [2]:
images, numbers = mnist.load_data()[0]

# Take only 20% of all samples from each class
idxs = []
for i in range(10):
    labels = np.squeeze(np.argwhere(numbers == i))
    end = len(labels) // 5
    idxs = np.append(idxs, labels[:end]).astype(int)

images = images[idxs]
numbers = numbers[idxs]

## Some data preprocessing
___

Let's reshape images into vectors and do the One Hot Encoding
for image labels:

In [3]:
# Images reshape
images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])

# One Hot Encoding
numbers = to_categorical(numbers)

## Dividing data on train and test datasets

In [4]:
train_img, test_img, train_num, test_num = train_test_split(images, numbers, test_size=.2)

## Model initialization and training

In [5]:
%%time

network = MLP(size=[784, 256, 256, 10], l_rate=.1)
network.train(train_img, train_num, epochs=2)

/home/pvl3p/Documents/Dev/Study/ML/misc/activation_functions.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


CPU times: user 4min 46s, sys: 2min 55s, total: 7min 41s
Wall time: 2min 1s


## Estimations

In [6]:
predicitions = np.array([np.squeeze(network.predict(x)) for x in test_img])

#### Accuracy:

In [7]:
accuracy(decode(test_num), decode(predicitions))

0.8066666666666666

## Classification using MLP from sklearn library

In [8]:
%%time

skl_mlp = MLPClassifier(hidden_layer_sizes=(256, 256),
                        activation="logistic",
                        alpha=0,
                        learning_rate_init=.01).fit(train_img, train_num)

CPU times: user 3min 14s, sys: 1min 30s, total: 4min 44s
Wall time: 1min 28s


## Estimations

In [9]:
skl_pred = skl_mlp.predict(test_img)

#### Accuracy:

In [10]:
accuracy(decode(test_num), decode(skl_pred))

0.775